# CAB330 Case Study 1: Students

Class: CAB330

Students: 

- Christopher Ayling | christopher.ayling@connect.qut.edu.au
- Benjamin Saljooghi | 9448233 | benjamin.saljooghi@connect.qut.edu.au
- Jordi Smit | 10294139 | jordi.smit@connect.qut.edu.au

Due Date: 9th September 2018

Project Demo: Week 8 Wednesday Lab

Weighting: 25%

---
## Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Manipulating Data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from collections import defaultdict

# Visualisations
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import SVG, Image
import graphviz
import pydot
from io import StringIO
from sklearn.tree import export_graphviz

# Algorithms
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import StandardScaler

In [ ]:
randomSeed = 330
np.random.seed(randomSeed)

---
## Data Loading

In [ ]:
students = pd.read_csv("./STUDENT.csv")
students.head()

rows, columns = students.shape; rows, columns

---
## Task 1. Data Selection and Distribution. (4 marks)

### Variable Descriptions

The following information would assist you in assigning the variables roles.

- There are three target variables namely, G1, G2 and G3, with different types. Choose the target that suits best according to the given task.
- Identify if the variable is an input variable or a supplementary variable.
- Data transformation is required for a few input variables to get improved accuracy

### 1.1 Proportion of Students Who Will Pass

In [ ]:
G3_counts = students['G3'].value_counts()
plt.pie(G3_counts, labels=G3_counts.index, startangle=90, shadow=True, explode=(0, 0.1), autopct='%1.1f%%')
plt.title("Proportion of Students Who Will Pass")
plt.axis('equal')
plt.show()

### 1.2 Data Cleaning

In [ ]:
cleaned = students.copy()

# impute age NAs with mean
cleaned['age'].fillna(cleaned['age'].mean(), inplace=True)

# impute reason NAs with unknown value because else it will be read as float instead of a string, which cause trouble with the hot one transformation.
cleaned['reason'].fillna("-", inplace=True)

# impute school NAs with unknown value because else it will be read as float instead of a string, which cause trouble with the hot one transformation..
cleaned['school'].fillna("-", inplace=True)

# drop unused columns
cleaned.drop(columns=["id", "InitialName"], inplace=True)

#TODO maby better to remove missing target variable?
# impute G1 NAs with mean
cleaned['G1'].fillna(cleaned['G1'].mean(), inplace=True)

# impute G2 NAs with mean
cleaned['G2'].fillna(cleaned['G2'].mean(), inplace=True)

#TODO Should we also check for outliers or inconsistencies?

### 1.3 Level of Measurement

In [ ]:
descriptions = {
    "id": ["student's id", False, False, 'nominal'],
    "InitialName": ["student's initial", False, False, 'nominal'],
    "school": ["student's school name", True, True, 'nominal'],
    "sex": ["student's sex", True, True, 'nominal'],
    "age": ["student's age", True, True, 'numerical'],
    "address": ["student's home address type", True, True, 'nominal'],
    "famsize": ["family size (≤ 3 or > 3)", True, True, 'ordinal'],
    "Pstatus": ["parent's cohabitation status (living together or apart)", True, True, 'nominal'],
    "Medu": ["mother’s education(0 – none, 1 – primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)", True, True, 'ordinal'],
    "Fedu": ["father’s education(0 – none, 1 – primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)", True, True, 'ordinal'],
    "Mjob": ["mother's job", True, True, 'nominal'],
    "Fjob": ["father's job", True, True, 'nominal'],
    "reason": ["reason to choose this school", True, True, 'nominal'],
    "guardian": ["student's guardian", True, True, 'nominal'],
    "traveltime": ["home to school travel time (1 – < 15 min., 2 – 15 to 30 min., 3 – 30 min. to 1 hour or 4 – > 1 hour)", True, True, 'ordinal'],
    "studytime": ["weekly study time (1 – < 2 hours, 2 – 2 to 5 hours, 3 – 5 to 10 hours or 4 – > 10 hours)", True, True, 'ordinal'],
    "failures": ["number of past class failures(n if 1 ≤ n < 3, else 4)", True, True, 'ordinal'],
    "schoolsup": ["extra educational school support (yes or no)", True, True, 'nominal'],
    "famsup": ["family educational support (yes or no)", True, True, 'nominal'],
    "paid": ["extra paid classes (yes or no)", True, True, 'nominal'],
    "activities": ["extra-curricular activities (yes or no)", True, True, 'nominal'],
    "nursery": ["attended nursery school (yes or no)", True, True, 'nominal'],
    "higher": ["wants to take higher education (yes or no)", True, True, 'nominal'],
    "internet": ["Internet access at home (yes or no)", True, True, 'nominal'],
    "romantic": ["with a romantic relationship (yes or no)", True, True, 'nominal'],
    "famrel": ["quality of family relationships (1 – very bad to 5 – excellent)", True, True, 'ordinal'],
    "freetime": ["free time after school (1 – very low to 5 – very high)", True, True, 'ordinal'],
    "goout": ["going out with friends (1 – very low to 5 – very high)", True, True, 'ordinal'],
    "Dalc": ["workday alcohol consumption (1 – very low to 5 – very high)", True, True, 'ordinal'],
    "Walc": ["weekend alcohol consumption (1 – very low to 5 – very high)", True, True, 'ordinal'],
    "health": ["current health status (1 – very bad to 5 – very good)", True, True, 'ordinal'],
    "absences": ["number of school absences (0 to 75)", True, True, 'numerical'],
    "G1": ["first period grade (0 to 20)", True, True, 'numerical', True],
    "G2": ["second period grade (0 to 20)", True, True, 'numerical', True],
    "G3": ["Final result (PASS/FAIL)", True, True, 'nominal', True],
}

red = 'background-color: Tomato'
green = 'background-color: MediumSeaGreen'
blue = 'background-color: DodgerBlue'
sblue = 'background-color: SlateBlue'
violet = 'background-color: Violet'
gray = 'background-color: Lightgray'
orange = 'background-color: Orange'

def highlight_useful(val):
    """ highlight True as green, false as red """
    return '' if val is None else green if val else red

def highlight_type(val):
    """ highlight based on variable type """
    if val == 'nominal':
        return blue
    elif val == 'ordinal':
        return sblue
    elif  val == 'numerical':
        return violet
    
def highlight_target(val):
    """ highlight if target variable """
    return orange if val else gray


COLUMN_INFORMATION = pd.DataFrame.from_dict(descriptions, orient='index')
COLUMN_INFORMATION.columns = ['Description', 'For Classification', 'For Regression', 'Variable Type', 'Target']
COLUMN_INFORMATION = COLUMN_INFORMATION[['Description', 'Target', 'Variable Type', 'For Classification', 'For Regression']]
COLUMN_INFORMATION['Target'] = COLUMN_INFORMATION['Target'].apply(lambda cell: bool(cell))

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
    display(
        COLUMN_INFORMATION.style.\
            applymap(highlight_useful, subset=['For Classification', 'For Regression']).\
            applymap(highlight_type, subset='Variable Type').\
            applymap(highlight_target, subset='Target')
    )

### 1.4 Distribution Scheme

In [ ]:
target_variables = ['G3']
features = cleaned.loc[:, cleaned.columns.difference(target_variables)]
targets = cleaned.loc[:, target_variables]

test_size = 0.2
training_size = 1.0 - test_size
X_training, X_test, Y_training, Y_test = train_test_split(features, targets, test_size=test_size, train_size=training_size, random_state=randomSeed, shuffle=False)

In [ ]:
accuracy_overview = {
    "decision_tree": {},
    "regression": {},
    "neural_network": {},
}

def format_accuracy_overview(Y, predicted_Y, most_important_features):
    return {
        "precision": precision_score(Y, predicted_Y),
        "recall": recall_score(Y, predicted_Y),
        "f1": f1_score(Y, predicted_Y),
        "accuracy": accuracy_score(Y, predicted_Y),
        "most_important_features": most_important_features
    }

---
## Task 2. Predictive Modeling Using Decision Trees

(4 marks)

### 2.1 Build a decision tree using default setting. 

In [ ]:
#Create a lable encoder for each of the nominal and ordinal features.
DT_label_encoders = {}
for name in descriptions:
    if ("nominal" in descriptions[name] or "ordinal" in descriptions[name]) and name in cleaned.columns.values.tolist():
        lb = preprocessing.LabelEncoder()
        #Check cleanded data for every possible class. If only done on training data it might miss some.
        lb.fit(cleaned[name].tolist())
        DT_label_encoders[name] = lb 

def transform_features(Data, encoders):
    """Transforms data based on the provided encoder"""
    Data_copy = Data.copy()
    for col_name in Data_copy.columns.values.tolist():
        if col_name in encoders:
            #Get encoder
            encoder = encoders[col_name]
            #Transform the data in this col
            col_values = Data_copy[col_name].tolist()
            Data_copy[col_name] = encoder.transform(col_values)
            
    return Data_copy

def transform_features_to_DT(Data):
    """Transforms the nominal and ordinal features into a format that can be compared by the decision tree"""
    return transform_features(Data, DT_label_encoders)

In [ ]:
#Transform the training data into a format with which the decision tree can work.
X_training_decision_tree_format = transform_features_to_DT(X_training) 
Y_training_decision_tree_format = transform_features_to_DT(Y_training) 


#Create a decision tree and train it on the formated training data.
dt = DecisionTreeClassifier(random_state=randomSeed)
dt.fit(X_training_decision_tree_format, Y_training_decision_tree_format)

#### a. What is the classification accuracy on training and test datasets?

In [ ]:
#Transform test data
X_test_decision_tree_format = transform_features_to_DT(X_test) 
Y_test_decision_tree_format = transform_features_to_DT(Y_test) 

#Create a dataframe to display the values.
training_score = dt.score(X_training_decision_tree_format, Y_training_decision_tree_format)
test_score = dt.score(X_test_decision_tree_format, Y_test_decision_tree_format)    
dt_preformance = pd.DataFrame([training_score, test_score], columns=target_variables, index=['Train accuracy', 'Test accuracy'])
dt_preformance

#### b. List the decision rules

In [ ]:
dt_features = X_training_decision_tree_format.columns.values.tolist()
dotfile = StringIO()
export_graphviz(dt, out_file=dotfile, feature_names=dt_features)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_png("default_tree.png") # saved in the following file - will return True if successful

The unique decision rules:
 - G2 <= 10.45
 - G2 <-=9.55
 - G1 <= 9.75
 - goout <=0.5
 - failures <= 0.5
 - sex <= 0.5
 - G2 <= 9.35
 - Medu <= 9.35
 - goout <= 3.5
 - Dalc < 3.5
 - G1 <= 9.85
 - age <= 17.5
 - Fedu <= 1.5
 - Fjob <= 0.5
 - studytime <= 1.0
 - age <= 16.868
 - guardian <= 1.5
 - absences <= 6.0
 - absences <= 9.0
 - G1 <= 11.225
 - famrel <= 2.5
 - walc <= 0.5
 - guardian <= 0.5
 - walc <= 2.5
 - dalc <= 0.5
 - Mjob <= 1.5
 - Medu <= 1.5
 - adress <= 0.5
 - Fedu <= 2.5
 - health <= 3.0
 - school <= 1.5
 - reason <= 2.5
 - freetime <= 2.5
 - G2 <= 11.35
 - G1 <= 11.55
 - traveltime <= 0.5
 - Fjob <= 1.5
 - reason <= 0.5
 - Walc <= 3.5
 - Walc <= 0.5
 - health <= 0.5
 - G1 <= 10.85
 - reason <= 1.5
 - paid <= 0.5
 - activities <= 0.5
 - nursery <= 0.5
 - G1 <= 10.55
 - famrel <= 3.5
 - G2 <= 11.55
 - G1 <= 12.85
 - schoolsup <= 0.5
 - absences <= 1.0
 - goout <= 05
 - Walc < 3.5

The complete decision tree:

![](default_tree.png)


#### c. What are the 5 important variables in building the tree?

In [ ]:
importances = dt.feature_importances_
importances_dt = pd.DataFrame(importances, index=dt_features, columns=["G3"])
importances_dt.nlargest(5, "G3")

#### d. Report if you see any evidence of model overfitting.

There is model overfitting on the training set since the decision tree has a 100% accuracy at the training set while it only has a 82.8% accuracy on the test set. This can also be seen in the tree since it has become extremely complicated with very many paths. This a classic example of overfitting.

### 2.2 Build another decision tree tuned with GridSearchCV.

In [ ]:
#The parameters to be searched
parameters = {
     "max_depth": [None, 1, 2, 3, 4, 5],
     "min_samples_split": [0.001, 0.005, 0.01, 0.05, 0.1],
     "min_samples_leaf": [1, 2, 4, 8, 16, 32],
     "criterion": ['gini', 'entropy'],
     "splitter" : ["best", "random"],
     "max_features": [None, "auto", "sqrt", "log2"],
     "max_leaf_nodes": [None, 2, 3, 4, 5, 6],
}
#creates and starts a grids search
gs_dt = GridSearchCV(DecisionTreeClassifier(random_state=randomSeed), parameters, n_jobs=8)
gs_dt.fit(X_training_decision_tree_format, Y_training_decision_tree_format)

#### a. What is the classification accuracy on training and test datasets?

In [ ]:
#Calc scores
training_score = gs_dt.score(X_training_decision_tree_format, Y_training_decision_tree_format)
test_score = gs_dt.score(X_test_decision_tree_format, Y_test_decision_tree_format)

#Display training and test score
dt_grid_preformance = pd.DataFrame([training_score, test_score], columns=["G3 grid"], index=['Train accuracy', 'Test accuracy'])
dt_grid_preformance

In [ ]:
dt_y_predicted = gs_dt.best_estimator_.predict(X_test_decision_tree_format)
dt_score_overview = classification_report(Y_test_decision_tree_format, dt_y_predicted)

#### b. What are the parameters used? Explain your decision.

The following parameters are being considered: 
 - **max_depth**: Limits the maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
 - **min_samples_split**: The minimum number of samples required to split an internal node.
 - **min_samples_leaf**: The minimum number of samples required to be at a leaf node.
 - **criterion**: The function to measure the quality of a split. Examples are Gini impurity and entropy/information gain.
 - **splitter**: The strategy used to choose the split at each node. Examples are best which choices the best criterion criteria or random which preforms a random split.
 - **max_features**:  Limits the number of features to consider when looking for the best split. For Example sqrt limits it to sqrt (total_number_of_features).
 - **max_leaf_nodes**:  Limits the number of leaf nodes. Nodes with relative reduction in impurity are added first to ensure the best possible tree with the constaint.
 
We decided to use the kitchen sink approach. We looked up the default values of the parameters and provided the grid search with a random range of values arround these default values. Then we the GridSearch run on multiple threads/jobs and we see which parameters return the optimal values.



#### c. What are the optimal parameters for this decision tree?

In [ ]:
print(f"The following parameters result in the best decision tree:")
for name in parameters:
    print(f"   -{name}: {gs_dt.best_params_[name]}")

#### d. Which variable is used for the first split? What are the competing splits for this first split?

In [ ]:
dt_features = X_training_decision_tree_format.columns.values.tolist()
dotfile = StringIO()
export_graphviz(gs_dt.best_estimator_, out_file=dotfile, feature_names=dt_features)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_png("grid_tree.png") # saved in the following file - will return True if successful


The first split is done based on the following rule: G2 <= 10.45. The split rules at the second level are G2 <= 9.55 & G2 <= 11.35.
Which is the same as the tree with the default parameter. The only different is that this tree has far less leaf nodes.

The result:

![](grid_tree.png)

#### e. What are the 5 important variables in building the tree?

In [ ]:
#Get the importances of each feature
importances = gs_dt.best_estimator_.feature_importances_

#display the 5 most important features
importances_dt_gs = pd.DataFrame(importances, index=dt_features, columns=["G3"])
importances_dt_gs
importances_dt_gs.nlargest(5, "G3")

In [ ]:
dt_5_most_imporant_features = importances_dt_gs.nlargest(5, "G3").index.values.tolist()
accuracy_overview["decision_tree"] = format_accuracy_overview(Y_test_decision_tree_format, dt_y_predicted, dt_5_most_imporant_features)

#### f. Report if you see an evidence of model overfitting.

There is no evidence of model overfitting, the accuracy on the train and test sets moved much closer to each other. The tree has also become much simpler compared to the original one as can be seen in the visualization image. This is also a good indicater that there is no overfitting.

### 2.3 What is the significant difference do you see between these two decision tree models? How do they compare performance-wise? Explain why those changes may have happened.

**What is the significant difference do you see between these two decision tree models?**
When you compare the visualization of the trees, the first thing you notice is that the grid tree is much smaller and simpler than the original one. 
The grid tree also does not have duplicated rules. The original thee had some duplicated rules in different branches. The original tree did this to fit the training data perfectly which resulted into over-fitting.

**How do they compare performance-wise**
The original tree fitted the training data perfectly  but only achieved an 82% accuracy on the test data due to over-fitting. While the grid tree only achieve a 93% accuracy on the training data, it achieved a 86% accuracy on the test data. Thus the grid tree is much better at generalizing, which result into a higher accuracy on data it has never seen before. Making the grid tree the better one.

**Explain why those changes may have happened**
These changes happen due to the grid search. Sklearning's grid search uses K-fold validation on each possible configuration. The K-fold validation prevents
 the training algorithm from over-fitting in combination with the configuration parameters. Eventually the search returns the configuration that has the highest K-fold score which result into a tree data is not over-fitted (assuming that the training data is representative).
In our case this resulted into a much smaller tree. A much smaller tree means that there are fewer rules and that the decision rules are much more general and have to focus on the most important features. This means that the tree cannot perfectly fit the training data and over-fit, which makes it preform better on data it has never seen before.

In [ ]:
comparision = pd.DataFrame([], columns=[])
print(comparision.append(dt_preformance["G3"]).append(dt_grid_preformance["G3 grid"]).transpose())

### 2.4 From the better model, can you identify which students to target for further consultation? Can you provide some descriptive summary of those students?

The following rules identify students that will most likely fail.

- `if G2 <= 9.55` There are 206 students in this group and 95.6% of them eventually failed. So this is the most important group to watch.
- `if 9.55 < G2 <= 10.45 & G1 <= 11.225` There are 73 students in this group and 72.6% of them eventually failed. So this is the second most important group to watch.

The following rules are far less interesting but still indicate students with a small changes of failing. 
- `if 10.45 < G2 <= 11.35 & G1 <= 11.55` There are 77 students in this group and 27.3% of them eventually failed.
- `if 9.55 < G2 <= 10.45 & G1 > 11.225` There are 16 students in this group and 18.8% of them eventually failed. 

As can be seen in the rules above only the grades provide a good indication about wheter the students will pass or not.

---
# Task 3. Predictive Modeling Using Regression

(5.5 marks)

## 3.1 Apply transformation/scaling methods to variables.

In [ ]:
def one_hot(data, column):
    dummies = pd.get_dummies(data[column], prefix=column)
    data = data.drop(column, axis=1)
    data = data.join(dummies)
    return data

def binary(data, column, map):
    new_col = data[column].map(map)
    data = data.drop(column, axis=1)
    data = data.join(new_col)
    return data
    
def transform(data):
    one_hot_cols = ['school', 'address', 'Mjob', 'Fjob', 'reason', 'guardian']
    yes_no_cols = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
    yes_no_map = {'no':0, 'yes':1}
    
    for col in one_hot_cols:
        data = one_hot(data, col)
    
    for col in yes_no_cols:
        data = binary(data, col, yes_no_map)
    
    data = binary(data, 'sex', {'F':0, 'M':1})
    data = binary(data, 'famsize', {'LE3':0, 'GT3':1})
    data = binary(data, 'Pstatus', {'A':0, 'T':1})
    data = binary(data, 'G3', {'FAIL':0, 'PASS':1})    
    return data

# get a copy of the data set and transform it
reg_data = transform(cleaned.copy())

# visualize transformed data
with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
    display(reg_data)

# split transformed data into features and targets
reg_features = reg_data.loc[:, reg_data.columns.difference(target_variables)]
reg_targets = reg_data.loc[:, target_variables]

# train test split the transformed data
reg_X_train_frame, reg_X_test_frame, reg_Y_train_frame, reg_Y_test_frame = train_test_split(reg_features, reg_targets, test_size=test_size, train_size=training_size, random_state=randomSeed, shuffle=False)
    
# convert to numpy matrices
reg_X_train = reg_X_train_frame.values
reg_X_test = reg_X_test_frame.values
reg_Y_train = reg_Y_train_frame.values
reg_Y_test = reg_Y_test_frame.values
# make Y arrays one-dimensional
reg_Y_train =  np.ravel(reg_Y_train)
reg_Y_test = np.ravel(reg_Y_test)

# apply scaling
scaler = StandardScaler()
# learn the mean and std.dev of variables from training data
# then use the learned values to transform training data
reg_X_train = scaler.fit_transform(reg_X_train, reg_Y_train)    
# transform the test data with the scaler that was trained on the training data
reg_X_test = scaler.transform(reg_X_test)


## 3.2 Build regression models 

In [ ]:
models = {
    "reg": None,
    "reg_cv": None,
    "reg_rfe": None,
    "reg_cv_rfe": None,
}

# Evaluation
def report_overfitting(model_name, X_train, X_test):
    print("Train accuracy:", models[model_name].score(X_train, reg_Y_train))
    print("Test accuracy:", models[model_name].score(X_test, reg_Y_test))
    
def report_accuracy(model_name, X_test):
    print(classification_report(reg_Y_test, models[model_name].predict(X_test)))
    
models["reg"] = LogisticRegression()
models["reg"].fit(reg_X_train, reg_Y_train)

params = {'C': [pow(10, x) for x in range(-6, 4)]}
models["reg_cv"] = GridSearchCV(LogisticRegression(), params, cv=10, n_jobs=-1)
models["reg_cv"].fit(reg_X_train, reg_Y_train)

### a. Report which variables are included in the regression model.

In [ ]:
feature_names = reg_X_train_frame.columns
print(feature_names)

### b. Report the top-5 important variables.

In [ ]:
coef = models["reg"].coef_[0]
indices = np.argsort(np.absolute(coef))
indices = np.flip(indices, axis=0)
top_5 = [(feature_names[i], coef[i]) for i in indices[:5]]
for feature, coefficient in top_5:
    print(feature, coefficient) 

### c. Report any sign of overfitting.

In [ ]:
print("Without GridSearchCV")
report_overfitting("reg", reg_X_train, reg_X_test)

print("\nWith GridSearchCV")
report_overfitting("reg_cv", reg_X_train, reg_X_test)

### d. What are the parameters used?

In [ ]:
# What are the parameters?
print("Parameters:")
print(models["reg"].get_params(), "\n")
    
# Explain your decision.
print("These parameters are used because they are the defaults.\n")

# What are the optimal parameters?
print("Optimal parameters:")
print(models["reg_cv"].best_params_, "\n")

# Which regression function is being used?
print("Logistic regression is used because we have a binary classification problem.")

### e. What is the classification accuracy on training and test datasets?

In [ ]:
print("Without GridSearchCV")
report_accuracy("reg", reg_X_test)

print("\nWith GridSearchCV")
report_accuracy("reg_cv", reg_X_test)

## 3.3 Build another regression model using the subset of inputs selected by RFE and selection by model methods

In [ ]:
rfe = RFECV(estimator = LogisticRegression(), cv=10)
rfe.fit(reg_X_train, reg_Y_train)
reg_X_train_sel = rfe.transform(reg_X_train)
reg_X_test_sel = rfe.transform(reg_X_test)

models["reg_rfe"] = LogisticRegression()
models["reg_rfe"].fit(reg_X_train_sel, reg_Y_train)

# hyperparameters = dict(C=np.logspace(0, 4, 10), penalty=['l1', 'l2'])
# models["reg_cv_rfe"] = GridSearchCV(LogisticRegression(), hyperparameters, cv=10, n_jobs=-1)
# models["reg_cv_rfe"].fit(reg_X_train_sel, reg_Y_train)
# models["reg_cv_rfe"] = LogisticRegressionCV()
# models["reg_cv_rfe"].fit(reg_X_train_sel, reg_Y_train)

params = {'C': [pow(10, x) for x in range(-6, 4)]}
models["reg_cv_rfe"] = GridSearchCV(LogisticRegression(), params, cv=10, n_jobs=-1)
models["reg_cv_rfe"].fit(reg_X_train_sel, reg_Y_train)

###  a. Report which variables are included in the regression model.

In [ ]:
for boolean, feature in zip(rfe.support_, feature_names):
    if boolean:
        print(feature)

### b. Report the top-5 important variables.

In [ ]:
rfe_sorted_features = []
for i in range(len(rfe.ranking_)):
    for index in range(len(rfe.ranking_)):
        rank = rfe.ranking_[index]
        if (rank == i+1):
            rfe_sorted_features.append(feature_names[index])
print(rfe_sorted_features[:5])

### c. Report any sign of overfitting.

In [ ]:
print("Without GridSearchCV")
report_overfitting("reg_rfe", reg_X_train_sel, reg_X_test_sel)

print("\nWith GridSearchCV")
report_overfitting("reg_cv_rfe", reg_X_train_sel, reg_X_test_sel)

### d. What is the classification accuracy on training and test datasets?

In [ ]:
print("Without GirdSearhCV")
report_accuracy("reg_rfe", reg_X_test_sel)

print("With GirdSearhCV")
report_accuracy("reg_cv_rfe", reg_X_test_sel)

## 3.4 Which of the regression models appears to be better?

In [ ]:
# Use comparison statistics.
print("Comparing all regression models f score, regression with RFE and GridsearchCV produces the best accuracy and f-score.")

# Is there any difference between two models (i.e one with selected variables and another with all variables)? 
print("Different accuracies, different number of variables, and different learned parameters.")

# Explain why those changes may have happened.
print("GridSearchCV determines the optimal hyperparameters which could influence the learned parameters. RFE eliminates most variables.")

## 3.5 From the better model, can you identify which students to target? 

In [ ]:
# Can you provide some descriptive summary of those students?
print("Good G1 and G2 score with low number of failures.")

In [ ]:
# Nominate RFE as best model
accuracy_overview["regression"] = format_accuracy_overview(reg_Y_test, models["reg_cv_rfe"].predict(reg_X_test_sel), rfe_sorted_features[:5])

print(accuracy_overview["regression"])

In [ ]:
pd.DataFrame.from_dict({"col1": [1, 2, 3]})

---
## Task 4. Predictive Modeling Using Neural Networks

(5.5 marks)

### 1. Build a Neural Network model using the default setting.

In [ ]:
Xtr_nn, ytr_nn = reg_X_train_array, reg_Y_train_array
Xte_nn, yte_nn = reg_X_test_array, reg_Y_test_array

In [ ]:
# Xtr_nn, ytr_nn = X_training_decision_tree_format, Y_training_decision_tree_format
# Xte_nn, yte_nn = X_test_decision_tree_format, Y_test_decision_tree_format

In [ ]:
goal = "G3"

In [ ]:
mlp = MLPClassifier()
mlp.fit(Xtr_nn, ytr_nn)

#### a. What is the network architecture of the model?

In [ ]:
print(f"the default neural network has {len(mlp.hidden_layer_sizes)} hidden layers(s) of size {mlp.hidden_layer_sizes}")

#### b. How many iterations are needed to train this network?

In [ ]:
print(f"the default neural network trained for {len(mlp.loss_curve_)} iterations")

#### c. Do you see any sign of over-fitting?

In [ ]:
mlp.score(Xte_nn, yte_nn)

In [ ]:
plt.plot(mlp.loss_curve_)

#### d. Did the training process converge and result in the best model?

In [ ]:
mlp.best_loss_, mlp.loss_

#### e. What is the classification accuracy on the training and test datasets?

In [ ]:
print(f"The classification on the training set compared to the test set is {mlp.score(Xtr_nn, ytr_nn)}/{mlp.score(Xte_nn, yte_nn)}")

### 2. Refine this network by refining is with GridSearchCV.

In this section a neural network's hyperparameters are tuned using GridSearchCV.

* `hidden_layer_size` was optimized to ensure that that a the complexity of the model was appropriate for the problem
* `activation` was optimized to ensure that the activation function of the model is capaable of representing the problm. Relu is max(0, x) and allows only relevant signals to pass through to the next layer. tanh is good when the sign of the signal is relevant but the magnitude needs to be limited. logistic is a mix of the two.

source: https://www.quora.com/How-should-I-choose-a-proper-activation-function-for-the-neural-network?share=1

In [ ]:
params = {
    "hidden_layer_sizes": [(50), (100), (100, 100)],
    "activation": ['logistic', 'tanh', 'relu'],
}

gs_mlp = GridSearchCV(MLPClassifier(max_iter=1000), params, n_jobs=-1).fit(Xtr_nn, ytr_nn)
plt.plot(mlp.loss_curve_)
display(
    f"train acc:{gs_mlp.score(Xtr_nn, ytr_nn)} test acc: {gs_mlp.score(Xte_nn, yte_nn)}",
    f"best loss: {gs_mlp.best_loss_} loss:{gs_mlp.loss_}",
    gs_mlp
)

### 3. Build  another Neural Network with inputs selected from RFE with regression.

(Use the best model generated in Task 3) and selection with decision tree (use the best model from Task 2).

In [ ]:
Xtr_nn_rfe, Xte_nn_rfe = reg_X_train_sel = rfe.transform(reg_X_train_array), rfe.transform(reg_X_test_array)
ytr_nn_rfe, yte_nn_rfe = ytr_nn, yte_nn

#### a. Did feature selection help here? Any changes in network architecture? What inputs are being used?

In [ ]:
mlp_rfe = MLPClassifier(max_iter=1000).fit(Xtr_nn_rfe, ytr_nn_rfe)

plt.plot(mlp_rfe.loss_curve_)
display(
    f"train acc:{mlp_rfe.score(Xtr_nn_rfe, ytr_nn_rfe)} test acc: {mlp_rfe.score(Xte_nn_rfe, yte_nn_rfe)}",
    f"best loss: {mlp_rfe.best_loss_} loss:{mlp_rfe.loss_}",
    mlp_rfe
)

#### b. What is the classification accuracy on the train and test datasets? Any improvements?

In [ ]:
f"train acc:{mlp_rfe.score(Xtr_nn_rfe, ytr_nn_rfe)} test acc: {mlp_rfe.score(Xte_nn_rfe, yte_nn_rfe)}"

#### c. How many iteration are needed to train this network?

The model was run for 100 iterations and converged at approximately 500 iterations.

#### d. Do you see any sign of over-fitting?

No, the difference between the train and test performance for the GridSearch/RFE model only approximately 4%, this is insignificant and is much lower than the 16% difference found in the pure GridSearch model.

#### e. Did the training process converge and result in the best model?

Yes, the training process converged.

#### f. Use GridSearchCV to tune the network to see whether the change in network architecture can further improve the performance.

In [ ]:
params = {
    "hidden_layer_sizes": [(50), (100), (100, 100)],
    "activation": ['logistic', 'tanh', 'relu'],
}

gs_mlp_rfe = GridSearchCV(MLPClassifier(max_iter=1000), params, n_jobs=-1).fit(Xtr_nn_rfe, ytr_nn_rfe)

plt.plot(gs_mlp_rfe.best_estimator_.loss_curve_)
display(
    f"train acc:{gs_mlp_rfe.score(Xtr_nn_rfe, ytr_nn_rfe)} test acc: {gs_mlp_rfe.score(Xte_nn_rfe, yte_nn_rfe)}",
    f"best loss: {gs_mlp_rfe.best_estimator_.best_loss_} loss:{gs_mlp_rfe.best_estimator_.loss_}",
    gs_mlp_rfe
)

### 3. Using the comparison methods, Which appears to be better?

From the better model, can you identify which customers to target? Can you provide some descriptive summary of those customers?

---
## Task 5. Comparing Predictive Models

(4 marks)

### 1. Using the comparison methods to compare the best decsision tree model, the best regression model...

#### a. Discuss the findings led by (a) ROC Chart and Index; (b) Accuracy Score; (c) Classification Report.

#### b. Do all the mdoels agree on the customers' characteristics? How do they vary?

### 2. Finally, based on all models and analysis, is there a particular model you will use in decision making?

### 3. Can you summarise positives and negaitives of each modelling method based on this analysis?

#### Decision tree
**Pros:**
 - Very easy to interpret. The model can be summarized into 2 simple if then rules. As can be seen in the image at 2.2.d.
 - Is very robust. It can handle most data types. Only catagorical data has to be encoded.
 - Relatively fast training time.

**Cons:**
 - Cannot handel complicated relationship. This is the reason why the accuracy is not higher than 86% on the test data.
 - Is very sensitive to overfitting. As can be seen in the default decision tree that had a 100% accuracy on the training set while it only had a 82% accuracy on the test set.

#### Regression
**Pros:**
 - List item
 - 

**Cons:**
 - List item
 - 

#### Neural Network
**Pros:**
 - List item
 - 

**Cons:**
 - List item
 - 

## Conclusion

In conclusion ...